In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import os



In [ ]:
driver = webdriver.Firefox()

# Change driver information to not detect bot
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_script("window.navigator.chrome = {runtime: {},  };")
driver.execute_script("window.navigator.permissions = {query: () => Promise.resolve({state: 'granted'}),  };")
driver.execute_script("const originalQuery = window.navigator.permissions.query; window.navigator.permissions.query = (parameters) => (parameters.name === 'notifications' ? Promise.resolve({state: Notification.permission}) : originalQuery(parameters));")

# Change user agent
driver.execute_script("window.navigator.userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'")


# Wait for page to load
time.sleep(5)

In [ ]:
table_links = []

for i in range(1,19):
    table_links.append(f'https://www.loc.gov/pictures/search/?sp={i}&co=ahii&st=grid')

print(table_links)

In [ ]:
image_page_urls = []

def get_images_from_page(url:str):
    driver.get(url)
    # Wait for page to load
    time.sleep(2)
    # Get page source
    html = driver.page_source
    # Parse html
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.find('tbody')
    
    if not tbody:
        # Thwor error
        raise Exception("No tbody found")
    
    tds = tbody.find_all('td')
    # Go every image page from td a href
    for td in tds:
        a = td.find('a')
        if a:
            href = a['href']
            print(href)
            image_page_urls.append(href)
            
for url in table_links:
    get_images_from_page(url)

In [ ]:
from lxml import etree 

image_name_urls = {}

def get_image_from_page(url:str):
    driver.get(url)
    # Wait for page to load
    time.sleep(2)
    # Get page source
    html = driver.page_source
    # Parse html
    soup = BeautifulSoup(html, 'html.parser')
    
    dom = etree.HTML(str(soup)) 

    # Get image Url with xpath
    a = dom.xpath("/html/body/div[1]/div[5]/div[3]/div/div[4]/p[2]/a[4]")[0]
    
    # Get href
    href = a.attrib['href']
    
    # Get image name
    name = dom.xpath('//*[@id="title"]')[0].text
    name = name.replace("[","").replace("]","").replace('\n', '').replace(' ', '_').replace('/', '_').replace(':', '_').replace("\t", "").replace("\r", "").replace("\xa0", "").replace(",", "").replace(".", "").replace("?", "").replace("!", "").replace(";", "").replace("(", "").replace(")", "").replace("'", "").replace('"', "").replace("’", "").replace("‘", "").replace("“", "").replace("”", "").replace("—", "").replace("–", "").replace("…", "")
    
    
    image_name_urls[name] = href
 

for url in image_page_urls:
    get_image_from_page(url)


In [ ]:

print(image_name_urls)
print(len(image_name_urls))

In [47]:
from alive_progress import alive_bar

# Create images folder
if not os.path.exists('images'):
    os.makedirs('images')
    

with alive_bar(len(image_name_urls)) as bar:
    for name, url in image_name_urls.items():
        # Get image
        response = requests.get(url)
        # Save image
        with open(f'images/{name}.tif', 'wb') as f:
            f.write(response.content)
        bar()

